<a href="https://colab.research.google.com/github/Min1222Ag/Digital-Watermarking/blob/model_architecture/model_skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Detection via YOLO

YOLO의 출력값을 bounding box의 좌표값(x_min,x_max,y_min,y_max)로 출력하도록 조정

In [ ]:
!pip install torch torchvision torchaudio
!pip install numpy matplotlib opencv-python Pillow pyyaml

# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt  # install requirements

In [ ]:
from google.colab import files
import os

# upload data
uploaded = files.upload()

In [ ]:
# 업로드된 파일을 훈련, 검증, 테스트 디렉토리로 분류
os.makedirs('/content/dataset/images/train', exist_ok=True)
os.makedirs('/content/dataset/images/val', exist_ok=True)
os.makedirs('/content/dataset/images/test', exist_ok=True)
os.makedirs('/content/dataset/labels/train', exist_ok=True)
os.makedirs('/content/dataset/labels/val', exist_ok=True)
os.makedirs('/content/dataset/labels/test', exist_ok=True)

# 예를 들어, 모든 이미지와 라벨이 한 폴더로 업로드된 것으로 가정하고 분류
# 실제 사용시 파일 명을 확인하여 분류 필요
import shutil
from sklearn.model_selection import train_test_split

# 파일 경로 목록 생성
image_paths = [x for x in uploaded.keys() if x.endswith('.jpg')]
label_paths = [x.replace('.jpg', '.txt') for x in image_paths]

# 훈련, 검증, 테스트 분할
train_files, val_files = train_test_split(list(zip(image_paths, label_paths)), test_size=0.2, random_state=42)
train_files, test_files = train_test_split(train_files, test_size=0.1, random_state=42)

# 파일을 적절한 폴더로 이동
def move_files(file_list, split_type):
    for img, label in file_list:
        shutil.move(img, f'/content/dataset/images/{split_type}/{img}')
        shutil.move(label, f'/content/dataset/labels/{split_type}/{label}')

move_files(train_files, 'train')
move_files(val_files, 'val')
move_files(test_files, 'test')


In [ ]:
data_yaml = f"""
train: /content/dataset/images/train
val: /content/dataset/images/val
test: /content/dataset/images/test

nc: 3
names: ['class1', 'class2', 'class3']
"""

with open('/content/yolov5/data/dataset.yaml', 'w') as f:
    f.write(data_yaml)

In [ ]:
%cd /content/yolov5

# 훈련 파라미터 설정
# --img: 입력 이미지 크기
# --batch: 배치 사이즈
# --epochs: 훈련 에포크 수
# --data: 데이터셋 설정 파일 경로
# --weights: 초기 가중치 파일 (여기서는 YOLOv5s 모델을 기반으로 시작)
# --cache: 이미지 캐시를 사용하여 훈련 속도 향상
# --patience: early stopping을 위한 인내심 값 (성능 개선이 없는 경우 훈련 중단)
# --img: 모델에 입력할 이미지의 크기
# --rect: 직사각형 훈련 사용 (비사각형 이미지에 유리)
# --multi-scale: 다양한 이미지 크기로 훈련을 진행하여 일반화 성능 향상
# --label-smoothing: 라벨 스무딩 적용 옵션 (과적합 방지)

!python train.py --img 640 --batch 16 --epochs 50 --data data/dataset.yaml --weights yolov5s.pt --cache --patience 7 --img 640 --rect --multi-scale --label-smoothing 0.1



In [ ]:
import torch
from PIL import Image
import cv2
from IPython.display import display

# 훈련된 모델 로드
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/yolov5/runs/train/exp/weights/best.pt', force_reload=True)

# 이미지 로드 및 예측 실행
img_path = '/content/sample_data/image.png'
results = model(img_path)

# 결과 출력
results.show()

# 바운딩 박스 추출
bboxes = results.xyxy[0]  # 바운딩 박스: [x_min, y_min, x_max, y_max, confidence, class]

# 워터마크 삽입 함수
def apply_watermark(image_path, bbox, watermark_text="Watermark"):
    image = cv2.imread(image_path)
    for (x_min, y_min, x_max, y_max, _, _) in bbox:
        x_min, y_min, x_max, y_max = map(int, [x_min, y_min, x_max, y_max])
        roi = image[y_min:y_max, x_min:x_max]
        cv2.putText(roi, watermark_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        image[y_min:y_max, x_min:x_max] = roi
    cv2.imwrite('watermarked_image.png', image)
    display(Image.open('watermarked_image.png'))

# 워터마크 적용
apply_watermark(img_path, bboxes)